# Notebook Description

This notebook shows how to simulate a two-dimensional shock-bubble interaction using MFC.
The simulation shows a helium bubble in air being impinged with a Mach 1.5 shockwave.
This is a canonical test problem in compressible fluid dynamics with ample experimental and computational results to compare to.
This notebook highlights how Cybershuttle can be used to dispatch software to multiple different systems from one notebook.

# Request Resources

In [ ]:
%pip install -q "airavata-python-sdk[notebook]==2.1.3.post2"
import airavata_jupyter_magic

%authenticate

# Request resource on SDSC Expanse
# %request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared

# Request resource on Jetstream 2
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud

%wait_for_runtime hpc_cpu --live
%copy_data source=local:case.py target=hpc_cpu:case.py 
%switch_runtime hpc_cpu

# Run Job on Expanse

In [ ]:
! export MFC_Root=/home/gridchem/apps/MFC/MFC-Wilfong; \
. $MFC_Root/mfc.sh load -c e -m c; \
module load cmake/3.21.4; \
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4 --no-build;

# Run on Jetstream 2

In [ ]:
! export MFC_Root="$HOME/apps/MFC-Wilfong"; \
module load gnu9/9.4.0 openmpi4/4.1.1; \
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4 --no-build;

# Compress Database Files For Transfer

In [ ]:
! tar -czf silo.tar.gz silo_hdf5/; \
echo "Tarball created"

# Transfer Files

In [ ]:
%copy_data source=hpc_cpu:silo.tar.gz target=local:silo.tar.gz